In [8]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
path = r'C:\Users\barka\OneDrive\Final Project\Data\Manhattan\2019\Model Learning'
less_than_10_df = pd.read_csv(path + '\less_than_10.csv')
beteween_10_and_15_df = pd.read_csv(path+ r'\between_10_and_15.csv')
more_than_15_df = pd.read_csv(path+'\more_than_15')
df_list = [less_than_10_df,beteween_10_and_15_df,more_than_15_df]

In [5]:
less_than_10_df.head()

,Zone,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,00:00 - 00:59,01:00 - 01:59,...,Tdep,HDD,CDD,Precipitation,new_snow,snow_depth,trip_distance,trip_time,speed,num_of_taxis
0,4,0,0,0,0,0,1,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.062857,8.428571,13.596331,14
1,4,0,0,0,0,0,1,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.560769,9.528205,14.936369,13
2,4,0,0,0,0,0,1,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.225000,9.031250,14.708362,8
3,4,0,0,0,0,0,1,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,2.399091,9.384848,14.133801,11
4,4,0,0,0,0,0,1,0,0,0,...,13.3,16.0,0.0,0.06,0.0,0.0,1.768333,7.234722,14.140612,12


In [3]:
df = less_than_10_df
y = df.num_of_taxis
X = df[df.columns.tolist()[:-1]]

In [7]:
data_dmaxtrix = xgb.DMatrix(data=X,label=y)

C:\Users\barka\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [10]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [13]:
xg_reg.fit(X_train,y_train)
train_pred = xg_reg.predict(X_train)
test_pred = xg_reg.predict(X_test)


C:\Users\barka\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:41:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


In [16]:
rmse_test = np.sqrt(mean_squared_error(y_test, test_pred))
rmse_train = np.sqrt(mean_squared_error(y_train,train_pred))
print("RMSE: %f" % (rmse_test))
print("RMSE: %f" % (rmse_train))

RMSE: 91.343622
RMSE: 90.874467
